# Tutorial

<b>Warning: it is necessary to use <u>setups='sg15'</u> in GPAW</b> (ONCV pseudopotentials)

In [56]:
from gpaw import GPAW, FermiDirac
from gpaw.wavefunctions.pw import PW
from ase.io import read
from TDDFT import TDDFT
import numpy as np
from tqdm import tqdm
from ase.units import Hartree, Bohr
import matplotlib.pyplot as plt
%matplotlib inline

c = 20
PW_cut=600
nbands=8
atoms = read('hBN.cif')
atoms.cell[2,2]=c
atoms.center()

calc = GPAW(mode=PW(PW_cut),xc='PBE',
            kpts={'size': (16, 16, 1), 'gamma': True},
            setups='sg15',nbands=nbands*2,
            occupations=FermiDirac(0.0001),
            convergence={'bands':nbands},
            txt='calc.txt')

atoms.set_calculator(calc)
atoms.get_potential_energy()
calc.write('gs_sg15.gpw')

# Initialization 

In [ ]:
tddft=TDDFT(calc,nbands)

 35%|███▌      | 46/130 [07:05<12:46,  9.13s/it]

# Calculation of the dipole matrix
Сalculation of the dipole matrix occurs according to the following equation
$$ d_{nm}(k)=\int_\Omega dr \; u_{kn}^{*}(r) \; r \; u_{km}(r)$$
where $u_{kn}(r)$ - periodic part of Kohn-Sham wavefunction which is stored in <b>TDDFT.ukn</b>

$r$ - coordinate inside the elementary cell which is stored in <b>TDDFT.r</b>

$\Omega$ - volume of the elementary cell which is stored in <b>TDDFT.volume</b>

In [ ]:
direction=[1,0,0]
dipole=tddft.get_dipole_matrix(direction)
plt.title("Dipole matrix at K=0")
plt.imshow(np.abs(np.sum(tddft.wk[:,None,None]*dipole,axis=0)))
plt.colorbar()

In [ ]:
plt.plot(np.sum(tddft.wk[:,None,None]*dipole,axis=0).diagonal(),'-o')

# Calculation of the Hartree matrix
Calculation of the Hartree matrix occurs according to following equation
$$ V^{H}_{nm}(k)=\int_\Omega dr \; u_{kn}^{*}(r) \; V_H(r) \; u_{km}(r)$$
$V_H(r)$ - Hartree potential which is obtained by solving Poisson equation using folowing equations:
$$n(G)=FFT(n(r)) \Rightarrow V_H(G)=-4\pi\frac{n(G)}{|G|^2} \Rightarrow V_H(r)=IFFT(V_H(G))$$
$$n(r)=2 \sum_{k}^{IBZ} w(k)\sum_n^{N_b}f_n\sum_m^{N_b} \left|c_{nm}(k)u_{km}(r)\right|^2$$
where $IBZ$ -irreducible Brillioun zone

$w(k)$ - weight of k-points in irreducible Brillioun zone which is stored in <b>TDDFT.w</b>

$f_n$ - occupation of Kohn-Sham orbitals which is stored in <b>TDDFT.f</b>

$c_{nm}(k)$ - wavefunction in second-quantization basis

In [ ]:
VH_matrix=tddft.get_Hartree_matrix()
plt.title("Hartree matrix at K=0")
plt.imshow(VH_matrix[0].real)
plt.colorbar()

# Calculation of the Fock matrix
Calculation of the Fock matrix occurs according to following equation
$$ V^{F}_{nm}(k)=-\sum^{occ}_l \sum_{q}^{BZ} \sum_{G} M^{*}_{ln}(k,q,G)M_{l,m}(k,q,G)v(q+G)$$
where $M_{n,m}(k,q,G)$ - pair-density which is stored in <b>TDDFT.M</b>
$$M_{n,m}(k,q,G)=IFFT\left(u_{k+q,n}^{*}(r)u_{k,n}(r)\right)$$
and $v(q+G)$ - Coloumb potential
$$v(q+G)=\frac{4\pi}{\left|q+G\right|^{2}}$$

In [ ]:
VF_matrix=tddft.get_Fock_matrix()
plt.imshow(VF_matrix[0].real)
plt.colorbar()

# Calculation of the LDA exchange matrix
Calculation of the LDA exchange matrix matrix occurs according to following equation
$$ V^{LDAx}_{nm}(k)=\int_\Omega dr \; u_{kn}^{*}(r) \; \left(-\frac{3n(r)}{\pi}\right)^{1/3} \; u_{km}(r)$$

In [ ]:
LDAx_matrix=tddft.get_LDA_exchange_matrix()
plt.imshow(LDAx_matrix[0].real)
plt.colorbar()

# Calculation of the LDA correlation matrix
Calculation of the LDA correlation matrix matrix occurs according to following equation
$$ V^{LDAx}_{nm}(k)=\int_\Omega dr \; u_{kn}^{*}(r) \; f_c(n(r)) \; u_{km}(r)$$
$f_c(n(r))$ parametrization taken from THE JOURNAL OF CHEMICAL PHYSICS 145, 157101 (2016)

In [ ]:
LDAc_matrix=tddft.get_LDA_correlation_matrix()
plt.imshow(LDAc_matrix[0].real)
plt.colorbar()

# TDDFT

In [ ]:
dt=0.1;steps=2000;
E=np.zeros(steps);
time=np.arange(steps)*dt
E=10e-4*np.exp(-time/dt)
result=tddft.propagate(dt=dt,steps=steps,E=E,operator=dipole)

In [ ]:
spectrum=np.fft.fft(tddft.macro_dipole-tddft.macro_dipole[0])
freq = np.fft.fftfreq(steps, d=dt)*2*np.pi
spectrum=spectrum[np.argsort(freq)]
freq=np.sort(freq)
spectrum=np.abs(spectrum*freq)**2

In [ ]:
# plt.plot(omega*Hartree,epsilon.imag)
plt.plot(freq,spectrum,'-')
# plt.plot(data[inds_w, 0], 4 * np.pi * data[inds_w, 4],label='real')
# plt.legend(loc='best')
plt.xlim([0,10])
plt.ylim([0,None])
plt.grid()

In [ ]:
plt.plot(time,tddft.macro_dipole)
# plt.plot(tddft.macro_dipole.imag)
plt.xlim([0,100])
plt.grid()

In [ ]:
np.mean(tddft.macro_dipole)

In [ ]:
tddft.macro_dipole[0]